In [ ]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
import xlsxwriter


In [ ]:
#Get the Coursera Catalogue from the API. While loop is in place for the pagination, to get the 'next' 100 rows.
  #Try to solve the universities with the url API
next = 0
total = 1
appended_data =[]
while next < (total):
    try:
        url = "https://api.coursera.org/api/courses.v1?start=" + str(next) +"&primaryLanguages,categories"
        course = requests.get(url)
        data = json.loads(course.text)
        #dframe = pd.DataFrame(data['elements']
        dframe = json_normalize(data['elements'])
        appended_data.append(dframe)
        #print (url) remove comment to see url's generated
        next = (int(data['paging']["next"]))
        total = (int(data['paging']["total"]))
        #print (total) remove comment to see 'totals' per page
        #print (next) remove comment to see 'nexts' per page
    except KeyError:
        break
DFCoursera = pd.concat(appended_data)

In [ ]:
#Generate the course_url in order to use it for the SharedCount API. Add column to the DFCoursera dataframe
DFCoursera['course_url'] = 'https://www.coursera.org/learn/' + DFCoursera['slug']

In [ ]:
#Function for the social media 'get'
private_api_key = 'key'
def get_social_metrics(url):
        sharedcount_response = requests.get('https://free.sharedcount.com/?url=' + url + '&apikey=' + private_api_key)
        data = sharedcount_response.text
        dataj = json.loads(data)
        datak = json_normalize(dataj)
        return datak
                                            

In [ ]:
#Get for every URL the SharedCount results and put these in a table.
appended_data = []
for url in DFCoursera['course_url']:
    data = get_social_metrics(url)
    appended_data.append(data) ## store dataframes in list
appended_data = pd.concat(appended_data, axis=0) ## concat elements of list


In [ ]:
#Renaming
DFSocial = appended_data

In [ ]:
#Delete index in order to concat
DFCoursera=DFCoursera.reset_index(drop=True)

In [ ]:
#Delete index in order to concat
DFSocial=DFSocial.reset_index(drop=True)

In [ ]:
#Concatenation
DFCoSo = pd.concat([DFCoursera, DFSocial], axis = 1)

In [ ]:
#Export to Excel
writer = pd.ExcelWriter('Coursera_allcourses.xlsx', engine='xlsxwriter')
DFCoSo.to_excel(writer, sheet_name='Sheet1')
writer.save()
